## Introduction to tensors

In [1]:
import numpy as np

In [2]:
class Tensor(object):
    
    def __init__(self,data,creators=None,creation_op=None):
        self.data = np.array(data)
        self.creators = creators
        self.creation_op= creation_op
        self.grad = None
    def backward(self,grad):
        self.grad = grad
        if(self.creation_op =="add"):
            self.creators[0].backward(grad)
            self.creators[1].backward(grad)
                
    def __add__(self,other):
        return Tensor(self.data+other.data,
                         creators=[self,other],
                        creation_op="add")
    
    def __repr__(self):
        return str(self.data.__repr__())
    def __str__(self):
        return str(self.data.__str__())

creating a tensor

In [3]:
x = Tensor([1,2,3,4,5])
print(x)

[1 2 3 4 5]


Perfomring addition operation with two tensors

In [4]:
y = x+x
print(y)

[ 2  4  6  8 10]


Right now there are no gradinets since we did not do a back propogation

In [5]:
print(x.grad)
print(y.grad)


None
None


Here we are adding two vectors to create z this should create a computation graph and have it pointed to z with nodes x and y which edges labeling the creation_op

In [6]:
z = x+y
print(z)
z.backward(Tensor([1,1,1,1,1]))

[ 3  6  9 12 15]


In [7]:
print(x.grad)
print(y.grad)

[1 1 1 1 1]
[1 1 1 1 1]


In [8]:
print(z.creators)

[array([1, 2, 3, 4, 5]), array([ 2,  4,  6,  8, 10])]


In [9]:
print(z.creation_op)

add


The most elegent part of this autograd is that it works recursively as well becuase each vector calls .backward() on all of its self.creators:

In [10]:
a = Tensor([1,2,3,4,5])
b = Tensor([2,2,2,2,2])
c = Tensor([5,4,3,2,1])
d = Tensor([-1,-2,-3,-4,-5])

e = a+b
f = c+d
g = e+f
g.backward(Tensor(np.array([1,1,1,1,1])))
listy=[a,b,c,d,e,f,g]
for i in listy:
    print(i,"has gradient",i.grad)

[1 2 3 4 5] has gradient [1 1 1 1 1]
[2 2 2 2 2] has gradient [1 1 1 1 1]
[5 4 3 2 1] has gradient [1 1 1 1 1]
[-1 -2 -3 -4 -5] has gradient [1 1 1 1 1]
[3 4 5 6 7] has gradient [1 1 1 1 1]
[ 4  2  0 -2 -4] has gradient [1 1 1 1 1]
[7 6 5 4 3] has gradient [1 1 1 1 1]


# Updating autograd to support multiuse tensor

In [11]:
import numpy as np


class Tensor(object):

    def __init__(self,data,
                autograd=False,
                creators=None,
                creation_op=None,
                id=None):
        self.data = np.array(data)
        self.creation_op = creation_op
        self.creators = creators
        self.grad = None
        self.autograd = autograd
        self.childern = {}

        if(id is None):
            id = np.random.randint(0,100000)
        self.id = id

        if(creators is not None):
            for c in creators:
                if(self.id not in c.childern):
                    c.childern[self.id] = 1
                else:
                    c.childern[self.id] += 1
    def all_childern_grads_accounted_for(self):
        for id,cnt in self.childern.items():
            if(cnt !=0 ):
                return False
        return True

    def backward(self,grad=None,grad_origin=None):
        if(self.autograd):
            if(grad_origin is not None):
                if(self.childern[grad_origin.id]==0):
                    raise Exception("cannto backprop more than once")
                else:
                    self.children[grad_origin.id] -= 1

        if(self.grad is None):
            self.grad = grad
        else:
            self.grad += grad
        if(self.creators is not None and
            (self.all_childern_grads_accounted_for()or grad_origin is None)):
                if(self.creation_op =="add"):
                    self.creators[0].backward(grad)
                    self.creators[1].backward(grad)

    def __add__(self,other):
        if(self.autograd and other.autograd):
            return Tensor(self.data+other.data,
                          autograd=True,
                      creators=[self,other],
                      creation_op="add")
        return Tensor(self.data+other.data)


    def __repr__(self):
        return str(self.data.__repr__())

    def __str__(self):
        return str(self.data.__str__())

In [12]:
a = Tensor([1,2,3,4,5],autograd=True)
b = Tensor([2,2,2,2,2],autograd=True)
c = Tensor([5,4,3,2,1],autograd=True)

In [13]:
d = a+b
e = b+c
f = d+e

f.backward(Tensor(np.array([1,1,1,1,1])))

print(b.grad.data == np.array([2,2,2,2,2]))

[ True  True  True  True  True]


## Adding support for negation

In [14]:
import numpy as np


class Tensor(object):

    def __init__(self,data,
                autograd=False,
                creators=None,
                creation_op=None,
                id=None):
        self.data = np.array(data)
        self.creation_op = creation_op
        self.creators = creators
        self.grad = None
        self.autograd = autograd
        self.childern = {}

        if(id is None):
            id = np.random.randint(0,100000)
        self.id = id

        if(creators is not None):
            for c in creators:
                if(self.id not in c.childern):
                    c.childern[self.id] = 1
                else:
                    c.childern[self.id] += 1
    def all_childern_grads_accounted_for(self):
        for id,cnt in self.childern.items():
            if(cnt !=0 ):
                return False
        return True

    def backward(self,grad=None,grad_origin=None):
        if(self.autograd):
            if(grad_origin is not None):
                if(self.childern[grad_origin.id]==0):
                    raise Exception("cannto backprop more than once")
                else:
                    self.children[grad_origin.id] -= 1

        if(self.grad is None):
            self.grad = grad
        else:
            self.grad += grad
        if(self.creators is not None and
            (self.all_childern_grads_accounted_for()or grad_origin is None)):
                if(self.creation_op =="add"):
                    self.creators[0].backward(grad)
                    self.creators[1].backward(grad)
                if(self.creation_op=="neg"):
                    self.creators[0].backward(self.grad.__neg__())

    def __add__(self,other):
        if(self.autograd and other.autograd):
            return Tensor(self.data+other.data,
                          autograd=True,
                      creators=[self,other],
                      creation_op="add")
        return Tensor(self.data+other.data)

    def __neg__(self):
        if(self.autograd):
            return Tensor(self.data * -1,
                          autograd=True,
                          creators=[self],
                          creation_op="neg")
        return Tensor(self.data * -1)

    def __repr__(self):
        return str(self.data.__repr__())

    def __str__(self):
        return str(self.data.__str__())

In [15]:
a = Tensor([1,2,3,4,5],autograd=True)
b = Tensor([2,2,2,2,2],autograd=True)
c = Tensor([5,4,3,2,1],autograd=True)

d = a +(-b)
e = (-b) + c
f = d+e

f.backward(Tensor(np.array([1,1,1,1,1])))

print(b.grad.data == np.array([-2,-2,-2,-2,-2]))

[ True  True  True  True  True]


## Adding additional functions

In [3]:
import numpy as np

class Tensor (object):

    def __init__(self,data,
                 autograd=False,
                 creators=None,
                 creation_op=None,
                 id=None):

        self.data = np.array(data)
        self.autograd = autograd
        self.grad = None
        if(id is None):
            self.id = np.random.randint(0,100000)
        else:
            self.id = id

        self.creators = creators
        self.creation_op = creation_op
        self.children = {}

        if(creators is not None):
            for c in creators:
                if(self.id not in c.children):
                    c.children[self.id] = 1
                else:
                    c.children[self.id] += 1

    def all_children_grads_accounted_for(self):
        for id,cnt in self.children.items():
            if(cnt != 0):
                return False
        return True

    def backward(self,grad=None, grad_origin=None):
        if(self.autograd):

            if(grad is None):
                grad = Tensor(np.ones_like(self.data))

            if(grad_origin is not None):
                if(self.children[grad_origin.id] == 0):
                    raise Exception("cannot backprop more than once")
                else:
                    self.children[grad_origin.id] -= 1

            if(self.grad is None):
                self.grad = grad
            else:
                self.grad += grad

            # grads must not have grads of their own
            assert grad.autograd == False

            # only continue backpropping if there's something to
            # backprop into and if all gradients (from children)
            # are accounted for override waiting for children if
            # "backprop" was called on this variable directly
            if(self.creators is not None and
               (self.all_children_grads_accounted_for() or
                grad_origin is None)):

                if(self.creation_op == "add"):
                    self.creators[0].backward(self.grad, self)
                    self.creators[1].backward(self.grad, self)

                if(self.creation_op == "sub"):
                    self.creators[0].backward(Tensor(self.grad.data), self)
                    self.creators[1].backward(Tensor(self.grad.__neg__().data), self)

                if(self.creation_op == "mul"):
                    new = self.grad * self.creators[1]
                    self.creators[0].backward(new , self)
                    new = self.grad * self.creators[0]
                    self.creators[1].backward(new, self)

                if(self.creation_op == "mm"):
                    c0 = self.creators[0]
                    c1 = self.creators[1]
                    new = self.grad.mm(c1.transpose())
                    c0.backward(new)
                    new = self.grad.transpose().mm(c0).transpose()
                    c1.backward(new)

                if(self.creation_op == "transpose"):
                    self.creators[0].backward(self.grad.transpose())

                if("sum" in self.creation_op):
                    dim = int(self.creation_op.split("_")[1])
                    self.creators[0].backward(self.grad.expand(dim,
                                                               self.creators[0].data.shape[dim]))

                if("expand" in self.creation_op):
                    dim = int(self.creation_op.split("_")[1])
                    self.creators[0].backward(self.grad.sum(dim))

                if(self.creation_op == "neg"):
                    self.creators[0].backward(self.grad.__neg__())

    def __add__(self, other):
        if(self.autograd and other.autograd):
            return Tensor(self.data + other.data,
                          autograd=True,
                          creators=[self,other],
                          creation_op="add")
        return Tensor(self.data + other.data)

    def __neg__(self):
        if(self.autograd):
            return Tensor(self.data * -1,
                          autograd=True,
                          creators=[self],
                          creation_op="neg")
        return Tensor(self.data * -1)

    def __sub__(self, other):
        if(self.autograd and other.autograd):
            return Tensor(self.data - other.data,
                          autograd=True,
                          creators=[self,other],
                          creation_op="sub")
        return Tensor(self.data - other.data)

    def __mul__(self, other):
        if(self.autograd and other.autograd):
            return Tensor(self.data * other.data,
                          autograd=True,
                          creators=[self,other],
                          creation_op="mul")
        return Tensor(self.data * other.data)

    def sum(self, dim):
        if(self.autograd):
            return Tensor(self.data.sum(dim),
                          autograd=True,
                          creators=[self],
                          creation_op="sum_"+str(dim))
        return Tensor(self.data.sum(dim))

    def expand(self, dim,copies):

        trans_cmd = list(range(0,len(self.data.shape)))
        trans_cmd.insert(dim,len(self.data.shape))
        new_data = self.data.repeat(copies).reshape(list(self.data.shape) + [copies]).transpose(trans_cmd)

        if(self.autograd):
            return Tensor(new_data,
                          autograd=True,
                          creators=[self],
                          creation_op="expand_"+str(dim))
        return Tensor(new_data)

    def transpose(self):
        if(self.autograd):
            return Tensor(self.data.transpose(),
                          autograd=True,
                          creators=[self],
                          creation_op="transpose")

        return Tensor(self.data.transpose())

    def mm(self, x):
        if(self.autograd):
            return Tensor(self.data.dot(x.data),
                          autograd=True,
                          creators=[self,x],
                          creation_op="mm")
        return Tensor(self.data.dot(x.data))

    def __repr__(self):
        return str(self.data.__repr__())

    def __str__(self):
        return str(self.data.__str__())

## Using autograd to train a neural network

Here we are doing our forward propogation by hand

In [17]:
import numpy as np
np.random.seed(1)

data = np.array([[0,0],[0,1],[1,0],[1,1]])
target = np.array([[0],[1],[0],[1]])

weights_0_1 = np.random.rand(2,3)
weights_1_2 = np.random.rand(3,1)

for i in range(10):
    layer_1 = data.dot(weights_0_1)
    layer_2 = layer_1.dot(weights_1_2)

    diff = (layer_2 - target)
    sqdiff = (diff * diff)
    loss = sqdiff.sum(0)

    layer_1_grad = diff.dot(weights_1_2.transpose())
    weights_1_2_update = layer_1.transpose().dot(diff)
    weights_0_1_update = data.transpose().dot(layer_1_grad)

    weights_1_2 -= weights_1_2_update * 0.1
    weights_0_1 -= weights_0_1_update * 0.1
    print(loss[0])

1.1205885886280245
0.9316716667584409
0.7837179804148104
0.6672343201081461
0.5729859059892466
0.49375101486194556
0.4249818890140674
0.3642093658828461
0.31016853006322104
0.2621751994107618


Now that we have our new autograd system we can use that to make the code much simpler


In [19]:
import numpy as np
np.random.seed(0)

data = Tensor(np.array([[0,0],[0,1],[1,0],[1,1]]), autograd=True)
target = Tensor(np.array([[0],[1],[0],[1]]), autograd=True)

w = []
w.append(Tensor(np.random.rand(2,3), autograd=True))
w.append(Tensor(np.random.rand(3,1), autograd=True))

for i in range(10):

    # Predict
    pred = data.mm(w[0]).mm(w[1])

    # Compare
    loss = ((pred - target)*(pred - target)).sum(0)

    # Learn
    loss.backward(Tensor(np.ones_like(loss.data)))

    for w_ in w:
        w_.data -= w_.grad.data * 0.1
        w_.grad.data *= 0

    print(loss)

[0.58128304]
[0.48988149]
[0.41375111]
[0.34489412]
[0.28210124]
[0.2254484]
[0.17538853]
[0.1324231]
[0.09682769]
[0.06849361]


# Adding automatic optimization

In [1]:
class SGD(object):
    def __init__(self,parameters,alpha=0.01):
        self.parameters = parameters
        self.alpha = alpha

    def zero(self):
        for p in self.parameters:
            p.grad.data *= 0
    def step(self,zero=True):
        for p in self.parameters:

            p.data -= p.grad.data * self.alpha

            if(zero):
                p.grad.data *= 0

The previous neural network is further simplified ad we can use this code now

In [4]:
import numpy as np
np.random.seed(0)

data = Tensor(np.array([[0,0],[0,1],[1,0],[1,1]]), autograd=True)
target = Tensor(np.array([[0],[1],[0],[1]]), autograd=True)

w = []
w.append(Tensor(np.random.rand(2,3), autograd=True))
w.append(Tensor(np.random.rand(3,1), autograd=True))
optim = SGD(parameters=w,alpha=0.01)
for i in range(10):

    # Predict
    pred = data.mm(w[0]).mm(w[1])

    # Compare
    loss = ((pred - target)*(pred - target)).sum(0)

    # Learn
    loss.backward(Tensor(np.ones_like(loss.data)))

    optim.step()


    print(loss)

[0.58128304]
[0.57031306]
[0.56002478]
[0.55027373]
[0.54094866]
[0.53196409]
[0.52325444]
[0.51476947]
[0.50647068]
[0.49832858]
